In [5]:
#Task1
!pip install python-chess
import chess

PIECE_VALUES = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 100
}

def evaluate_board(board):

    score = 0
    center_squares = [chess.D4, chess.D5, chess.E4, chess.E5]

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = PIECE_VALUES.get(piece.piece_type, 0)
            mobility = len(list(board.legal_moves)) * 0.1
            center_control = 0.5 if square in center_squares else 0

            if piece.color == chess.WHITE:
                score += value + mobility + center_control
            else:
                score -= value + mobility + center_control

    return score

def beam_search_chess(board, beam_width=3, depth_limit=3):
    beam = [(board, [], evaluate_board(board))]

    for _ in range(depth_limit):
        next_states = []

        for state, move_seq, _ in beam:
            legal_moves = list(state.legal_moves)

            for move in legal_moves:
                new_board = state.copy()
                new_board.push(move)
                score = evaluate_board(new_board)
                next_states.append((new_board, move_seq + [move], score))

        next_states.sort(key=lambda x: x[2], reverse=True)
        beam = next_states[:beam_width]

    best_moves, best_score = beam[0][1], beam[0][2]

    final_board = board.copy()
    for move in best_moves:
        final_board.push(move)

    return best_moves, best_score, final_board

board = chess.Board()

best_moves, best_score, final_board = beam_search_chess(board, beam_width=3, depth_limit=3)


print("Best Move Sequence:", [str(move) for move in best_moves])
print("Final Evaluation Score:", best_score)



Best Move Sequence: ['e2e4', 'f7f5', 'e4f5']
Final Evaluation Score: 2.8999999999999932


In [2]:
#Task2
import random
import math

def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def total_distance(route, locations):

    distance = sum(euclidean_distance(locations[route[i]], locations[route[i+1]]) for i in range(len(route)-1)) #dist from start till end
    distance += euclidean_distance(locations[route[-1]], locations[route[0]])  #to start
    return distance

def hill_climb_shortest_route(locations, max_iterations=1000):
    num_locations = len(locations)
    current_route = list(range(num_locations))
    random.shuffle(current_route)
    current_distance = total_distance(current_route, locations)

    for _ in range(max_iterations):

        i, j = random.sample(range(num_locations), 2)
        new_route = current_route[:]
        new_route[i], new_route[j] = new_route[j], new_route[i]

        new_distance = total_distance(new_route, locations)

        if new_distance < current_distance:
            current_route = new_route
            current_distance = new_distance

    return current_route, current_distance


locations = [(0, 0), (2, 3), (5, 4), (1, 2), (6, 8), (3, 7)]
best_route, best_distance = hill_climb_shortest_route(locations)

print("Optimized Route (Order of Indices):", best_route)
print("Total Distance Covered:", best_distance)


Optimized Route (Order of Indices): [2, 4, 5, 3, 0, 1]
Total Distance Covered: 21.6744450060527


In [15]:
import random

NUM_CITIES = 10
POPULATION_SIZE = 20
MUTATION_RATE = 0.3
MAX_GENERATIONS = 50
STALL_LIMIT = 5

def generate_distance_matrix(n):
    matrix = [[0 if i == j else random.randint(10, 100) for j in range(n)] for i in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            matrix[j][i] = matrix[i][j]
    return matrix

def calculate_route_distance(route, distance_matrix):
    return sum(distance_matrix[route[i]][route[i+1]] for i in range(len(route) - 1)) + distance_matrix[route[-1]][route[0]]

def create_random_individual():
    return random.sample(range(NUM_CITIES), NUM_CITIES)

def select_parents(population, distance_matrix):
    tournament_size = 5
    parents = []
    for _ in range(len(population) // 2):
        tournament = random.sample(population, tournament_size)
        best = min(tournament, key=lambda x: calculate_route_distance(x, distance_matrix))
        parents.append(best)
    return parents

def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [-1] * size
    child[start:end] = parent1[start:end]

    parent2_genes = [gene for gene in parent2 if gene not in child]
    idx = 0
    for i in range(size):
        if child[i] == -1:
            child[i] = parent2_genes[idx]
            idx += 1
    return child

def mutate(individual):
    if random.random() < MUTATION_RATE:
        i, j = random.sample(range(NUM_CITIES), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

def genetic_algorithm():
    distance_matrix = generate_distance_matrix(NUM_CITIES)
    population = [create_random_individual() for _ in range(POPULATION_SIZE)]

    best_distance = float('inf')
    stall_counter = 0

    for generation in range(MAX_GENERATIONS):
        population.sort(key=lambda ind: calculate_route_distance(ind, distance_matrix))
        current_best = population[0]
        current_best_distance = calculate_route_distance(current_best, distance_matrix)

        print(f"Generation {generation}: Best Distance = {current_best_distance}")

        if current_best_distance < best_distance:
            best_distance = current_best_distance
            best_individual = current_best[:]
            stall_counter = 0
        else:
            stall_counter += 1

        if stall_counter >= STALL_LIMIT:
            print("Stopping early due to no improvement in", STALL_LIMIT, "generations.")
            break

        parents = select_parents(population, distance_matrix)

        new_population = []
        while len(new_population) < POPULATION_SIZE:
            p1, p2 = random.sample(parents, 2)
            child = crossover(p1, p2)
            new_population.append(mutate(child))

        population = new_population

    return best_individual, best_distance

best_route, min_distance = genetic_algorithm()
print("\nOptimal Route:", best_route)
print("Minimum Distance:", min_distance)


Generation 0: Best Distance = 349
Generation 1: Best Distance = 318
Generation 2: Best Distance = 299
Generation 3: Best Distance = 278
Generation 4: Best Distance = 278
Generation 5: Best Distance = 259
Generation 6: Best Distance = 252
Generation 7: Best Distance = 252
Generation 8: Best Distance = 252
Generation 9: Best Distance = 252
Generation 10: Best Distance = 252
Generation 11: Best Distance = 252
Stopping early due to no improvement in 5 generations.

Optimal Route: [2, 7, 5, 4, 1, 9, 8, 0, 3, 6]
Minimum Distance: 252
